In [14]:
!pip install --upgrade keras
!pip install vidaug

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



In [15]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import tensorflow as tf  # for data preprocessing only
import keras
from keras import layers, ops, regularizers
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

In [1]:
# Reset the plotly graph
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Initialize plotly notebook mode
init_notebook_mode(connected=True)

# Load data (PKL)

In [16]:
import pickle
with open('/kaggle/input/3july-2-pkll/3July_2/ab_videos.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
ab_videos=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    ab_videos.append(array_a_loaded)
  except:
    print('end!')
    break
    
with open('/kaggle/input/3july-2-pkll/3July_2/nor_videos.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
nor_videos=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    nor_videos.append(array_a_loaded)
  except:
    print('end!')
    break
    
    
with open('/kaggle/input/3july-2-pkll/3July_2/ab_video_dims.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
ab_video_dims=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    ab_video_dims.append(array_a_loaded)
  except:
    print('end!')
    break
with open('/kaggle/input/3july-2-pkll/3July_2/nor_video_dims.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
nor_video_dims=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    nor_video_dims.append(array_a_loaded)
  except:
    print('end!')
    break

end!
end!
end!
end!


In [17]:
# 24 July
# Setting seed for reproducibility
SEED = 77
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
tf.random.set_seed(SEED)

# DATA
BATCH_SIZE = 4
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (40, 224, 224, 3)
NUM_CLASSES = 2

# OPTIMIZER
LEARNING_RATE = 1e-4 
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 40

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 64
NUM_HEADS = 2
NUM_LAYERS = 2

In [18]:
human_action_dataset = np.asarray( ab_videos + nor_videos )
# ab = 1
# nor = 0
labels = np.concatenate([np.ones(len(ab_videos)),np.zeros(len(nor_videos))])

ab_videos_dims_df = pd.DataFrame(ab_video_dims,columns=['frames','width','height','channel'])
nor_videos_dims_df = pd.DataFrame(nor_video_dims,columns=['frames','width','height','channel'])

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(human_action_dataset,labels,test_size=0.2,shuffle=True,random_state=42,stratify=labels)


# Noise Data
1. Data prepared in test and train containing a bunch of video
2. So, I need use for loop take out one by one then convert then into a list only can pass to the vidaug params 
3. Take notes that this process only involved test dataset
4. Those simulated test data will be proceed used in the next model (VP-GRU & CNN-GRU)

## Prepare Adding Brightnes dataset

In [6]:
from vidaug import augmentors as va

sometimes = lambda aug: va.Sometimes(0.3, aug) # Used to apply augmentor with 50% probability

chooseOne = va.Sequential([
    va.Add(30)
])


# load test set
ad_X_test = []
ad_y_test = []

for x, y in zip(X_test, y_test):
    new_x = chooseOne(x)
    ad_X_test.append(new_x)
    ad_y_test.append(y)

## Prepare Reducing brightness dataset

In [11]:
from vidaug import augmentors as va

chooseOne = va.Sequential([
    va.Add(-30)
])

# load test set
red_X_test = []
red_y_test = []

for x, y in zip(X_test, y_test):
    new_x = chooseOne(x)
    red_X_test.append(new_x)
    red_y_test.append(y)

## Prepare pepper noise dataset

In [12]:
from vidaug import augmentors as va
np.float = np.float64

chooseOne = va.Sequential([
    va.Pepper(10)
])

# load test set
pep_X_test = []
pep_y_test = []

for x, y in zip(X_test, y_test):
    new_x = chooseOne(x)
    pep_X_test.append(new_x)
    pep_y_test.append(y)

## Prepare gaussian blur dataset

In [13]:
from vidaug import augmentors as va

chooseOne = va.Sequential([
    va.GaussianBlur(1)
])

# load test set
gau_X_test = []
gau_y_test = []

for x, y in zip(X_test, y_test):
    new_x = chooseOne(x)
    gau_X_test.append(new_x)
    gau_y_test.append(y)

# ViViT Preprocess

In [19]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """
      Preprocess the frames tensors and parse the labels.
      This code convert the framess into float for normalization hence
      we should add a convert dtype code back to the model
    """
    # Preprocess images
    frames = tf.image.convert_image_dtype(frames[..., tf.newaxis],  tf.float32,)
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

# prepare test loader
# testloader_add = prepare_dataloader(ad_X_test, ad_y_test,'test')
# X_train,X_test,y_train,y_test
testloader = prepare_dataloader(X_test, y_test,'test')
# testloader_reduc = prepare_dataloader(red_X_test, red_y_test,'test')
# testloader_pep = prepare_dataloader(pep_X_test, pep_y_test,'test')
# testloader_gau = prepare_dataloader(gau_X_test, gau_y_test,'test')

In [ ]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm

BATCH_SIZE = 32  # Define your batch size here

@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """
    Preprocess the frames tensors and parse the labels.
    This code converts the frames into float for normalization hence
    we should add a convert dtype code back to the model.
    """
    # Preprocess images
    frames = tf.image.convert_image_dtype(frames[..., tf.newaxis], tf.float32)
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label

def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

def iterate_with_progress(dataloader):
    """Iterate over dataloader with tqdm progress bar."""
    num_batches = tf.data.experimental.cardinality(dataloader).numpy()
    progress_bar = tqdm(dataloader, total=num_batches, desc='Batches', unit='batch')
    
    for batch in progress_bar:
        # Perform your processing here
        pass  # Replace with actual processing logic

# Prepare test loader
# testloader_add = prepare_dataloader(ad_X_test, ad_y_test, 'test')

# Iterate over test loader with progress bar
iterate_with_progress(prepare_dataloader(ad_X_test, ad_y_test, 'test'))


In [20]:
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable(package="Custom", name="TubeletEmbedding")
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim, # embed_dim = 64
            kernel_size=patch_size, # patch_size = (8, 8, 8)
            strides=patch_size, 
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

@register_keras_serializable(package="Custom", name="PositionalEncoder")
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape # input_shape =(numVid,3920,64)
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1) # Set positions

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions # Concat the position with it token
        return encoded_tokens

# ViTPose preprocess

In [2]:
# Install ViT-Pose Framework
!git clone https://github.com/JunkyByte/easy_ViTPose.git
!cd easy_ViTPose/ && pip install -r requirements.txt && pip install -e . 
# Install ViT-Posepretrain weight
!pip install huggingface_hub 


Cloning into 'easy_ViTPose'...
remote: Enumerating objects: 924, done.
remote: Counting objects: 100% (340/340), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 924 (delta 242), reused 285 (delta 198), pack-reused 584
Receiving objects: 100% (924/924), 9.84 MiB | 25.97 MiB/s, done.
Resolving deltas: 100% (549/549), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.9/151.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 10.2 MB/s eta 0:00:00


In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

MODEL_SIZE = 'b'  #@param ['s', 'b', 'l', 'h']
YOLO_SIZE = 's'  #@param ['s', 'n']
DATASET = 'coco'  #@param ['coco_25', 'coco', 'wholebody', 'mpii', 'aic', 'ap10k', 'apt36k']
ext = '.pth'
ext_yolo = '.pt'

import os
from huggingface_hub import hf_hub_download
MODEL_TYPE = "torch"
YOLO_TYPE = "torch"
REPO_ID = 'JunkyByte/easy_ViTPose'
FILENAME = os.path.join(MODEL_TYPE, f'{DATASET}/vitpose-' + MODEL_SIZE + f'-{DATASET}') + ext
FILENAME_YOLO = 'yolov8/yolov8' + YOLO_SIZE + ext_yolo

print(f'Downloading model {REPO_ID}/{FILENAME}')
model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME) # ViT-Pose - Human Pose
yolo_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME_YOLO) # YOLO - Bouding box
# Load model
# If you get an error from PIL restart environment and rerun this cell to update packages version
from easy_ViTPose import VitInference
model_pose2 = VitInference(model_path, yolo_path, MODEL_SIZE,
                     dataset=DATASET, yolo_size=320, is_video=False)

In [ ]:
# Extract pose
# Trim video to a given frames (40 frames)
def trim_video_frames(video,max_frame):
    f,_,_,_ = video.shape
    startf = f//2 - max_frame//2
    return video[startf:startf+max_frame, :, :, :]

def prepare_data(X_test, y_test):
#     videos_path = df['file_path'].values.tolist()
#     labels = df['label'].to_list()
    
    pos_frames = [] # keeped all the video's pose
    updated_label = []
    for video,label in zip(X_test,y_test):
        count_success_frame=0 # Trigger unconsistent issues
        frames=[]
        for frame in video:
            # Extract keypoint using Vit-Pose
            frame_keypoints = model_pose2.inference(frame)
            if 0 in frame_keypoints: # Validate item in frame_keypoints
                frames.append(frame_keypoints[0][:,:2])
                count_success_frame+=1

        # Make sure each frame had contain human pose
        if count_success_frame==40:
            updated_label.append(label)
            pos_frames.append(frames)
        
    return pos_frames, updated_label

add_test_data, add_test_labels = prepare_data(ad_X_test, ad_y_test)
red_test_data, red_test_labels = prepare_data(red_X_test, red_y_test)
pep_test_data, pep_test_labels = prepare_data(pep_X_test, pep_y_test)
gau_test_data, gau_test_labels = prepare_data(gau_X_test, gau_y_test)


# Inception V3 preprocess

In [5]:
import os

import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2

IMG_SIZE = 224 # change
BATCH_SIZE = 64
EPOCHS = 80 # change default is 40

MAX_SEQ_LENGTH = 40 # change
NUM_FEATURES = 2048
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def prepare_all_videos(X_test, y_test):
    num_samples = len(X_test)
    labels = y_test

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, frames in enumerate(X_test):
        # Gather all its frames and add a batch dimension.
        frames = np.array(frames)
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(
            shape=(
                1,
                MAX_SEQ_LENGTH,
            ),
            dtype="bool",
        )
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :], verbose=0,
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


add_test_data_incep, add_test_labels_incep = prepare_all_videos(ad_X_test, ad_y_test)
red_test_data_incep, red_test_labels_incep = prepare_all_videos(red_X_test, red_y_test)
pep_test_data_incep, pep_test_labels_incep = prepare_all_videos(pep_X_test, pep_y_test)
gau_test_data_incep, gau_test_labels_incep = prepare_all_videos(gau_X_test, gau_y_test)

# ViViT Predction

In [21]:
model = tf.keras.models.load_model('/kaggle/input/vivit-24july-model/ViViT_24Jul.keras')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning:

`build()` was called on layer 'tubelet_embedding', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.



## Add Brightness

In [12]:
# Get the predictions
predictions = model.predict(testloader_add)

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [17]:
# Get the true labels
true_labels = np.concatenate([y for x, y in testloader_add], axis=0)
predicted_labels = np.argmax(predictions, axis=1)
z = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels)

# Plot matrix
import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()


## Reduced Brightness

In [12]:
# Get the predictions
predictions = model.predict(testloader_reduc)

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [16]:
# Get the true labels
true_labels = np.concatenate([y for x, y in testloader_reduc], axis=0)

predicted_labels = np.argmax(predictions, axis=1)
z = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels)

# Plot matrix
import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

## Pepper noise

In [12]:
# Get the predictions
predictions = model.predict(testloader_pep)

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [18]:
# Get the true labels
true_labels = np.concatenate([y for x, y in testloader_pep], axis=0)

predicted_labels = np.argmax(predictions, axis=1)
z = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels)

# Plot matrix
import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

## Gaussian blur

In [12]:
# Get the predictions
predictions = model.predict(testloader_gau)

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [19]:
# Get the true labels
true_labels = np.concatenate([y for x, y in testloader_gau], axis=0)

# Get the predictions
predictions = model.predict(testloader_gau)
# predictions = model.predict(validloader)
predicted_labels = np.argmax(predictions, axis=1)
z = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels)

# Plot matrix
import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

# VP-GRU

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/vivit-24july-model/VP-GRU_24Jul.keras')

## Add Brightness

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px

def calculate_metrics(model, X_test, y_test):
    # Get the true labels and predictions
    y_true = []
    y_pred = []
    label_dict = {0: 'Normal', 1: 'Abnormal'}

    for frame, label in zip(X_test,y_test):
        pred = model.predict(tf.expand_dims(frame, axis=0))[0]
        y_true.append(label)
        y_pred.append(np.argmax(pred))

    # confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    return cm


z = calculate_metrics(model,add_test_data,add_test_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [20]:
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)

## Reduced Brightness

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px

def calculate_metrics(model, X_test, y_test):
    # Get the true labels and predictions
    y_true = []
    y_pred = []
    label_dict = {0: 'Normal', 1: 'Abnormal'}

    for frame, label in zip(X_test,y_test):
        pred = model.predict(tf.expand_dims(frame, axis=0))[0]
        y_true.append(label)
        y_pred.append(np.argmax(pred))

    # confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    return cm


z = calculate_metrics(model,red_test_data,red_test_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [21]:
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)

## Pepper noise

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px

def calculate_metrics(model, X_test, y_test):
    # Get the true labels and predictions
    y_true = []
    y_pred = []
    label_dict = {0: 'Normal', 1: 'Abnormal'}

    for frame, label in zip(X_test,y_test):
        pred = model.predict(tf.expand_dims(frame, axis=0))[0]
        y_true.append(label)
        y_pred.append(np.argmax(pred))

    # confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    return cm


z = calculate_metrics(model,pep_test_data,pep_test_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [22]:
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)

## Gaussian blur

In [28]:
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px

def calculate_metrics(model, X_test, y_test):
    # Get the true labels and predictions
    y_true = []
    y_pred = []
    label_dict = {0: 'Normal', 1: 'Abnormal'}

    for frame, label in zip(X_test,y_test):
        pred = model.predict(tf.expand_dims(frame, axis=0))[0]
        y_true.append(label)
        y_pred.append(np.argmax(pred))

    # confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    return cm


z = calculate_metrics(model,gau_test_data,gau_test_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━

In [23]:
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)

# CNN-GRU

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/vivit-24july-model/CNN-GRU_24Jul.keras')

## Add Brightness

In [12]:
import plotly.graph_objects as go
import numpy as np
import tensorflow as tf

# Assuming `test_data` and `test_labels` are already defined
def generate_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Generate predictions
predicted_labels = generate_predictions(model, [add_test_data_incep[0], add_test_data_incep[1]])

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [24]:
z = tf.math.confusion_matrix(labels=add_test_labels_incep, predictions=predicted_labels)

import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

## Reduced Brightness

In [12]:
import plotly.graph_objects as go
import numpy as np
import tensorflow as tf

# Assuming `test_data` and `test_labels` are already defined
def generate_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Generate predictions
predicted_labels = generate_predictions(model, [red_test_data_incep[0], red_test_data_incep[1]])

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [25]:
z = tf.math.confusion_matrix(labels=red_test_labels_incep, predictions=predicted_labels)

import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

## Pepper noise

In [12]:
import plotly.graph_objects as go
import numpy as np
import tensorflow as tf

# Assuming `test_data` and `test_labels` are already defined
def generate_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Generate predictions
predicted_labels = generate_predictions(model, [pep_test_data_incep[0], pep_test_data_incep[1]])

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [26]:
z = tf.math.confusion_matrix(labels=pep_test_labels_incep, predictions=predicted_labels)

import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

## Gaussian blur

In [12]:
import plotly.graph_objects as go
import numpy as np
import tensorflow as tf

# Assuming `test_data` and `test_labels` are already defined
def generate_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Generate predictions
predicted_labels = generate_predictions(model, [gau_test_data_incep[0], gau_test_data_incep[1]])

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step


In [27]:
z = tf.math.confusion_matrix(labels=gau_test_labels_incep, predictions=predicted_labels)

import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()